# Modules and scoping rules

## What is a module?

Modules are used to organize larger Python projects.

A module is a file containing code. It defines a group of Python functions or other objects under a name which is derived from the name of the file.

Modules help avert name-clash related problems. With modules you can have two functions with the same name, which you will refer as:

```python
module1.my_fn

module2.my_fn
```

Each module creates its own namespace, which is essentially a dictionary of the identifiers available to each block, function, class, module, etc. 

The following block defines a simple module `mymath.py`:

```python
"""mymath - an example math module"""
pi = 3.14159

def area(r):
    return pi * r * r
```

As modules can contained compiled C or C++ code, the `.py` suffix is strongly recommended to let everyone understand the the file consists of Python code.

Importing the module will bring in the function and literal defined in the module.

You can do:

```python
import mymath
```

But in order to use the definitions for `pi` and `area()` you will have to qualify them with the module's name:

In [3]:
import mymath

try:
    print(pi)
except Exception as e:
    print("Oops! ", {e})

print(mymath.pi)
print(mymath.area(5))

Oops!  {NameError("name 'pi' is not defined")}
3.14159
78.53975


This form of access is known as qualification and guarantees name safety:

In [5]:
import math
import mymath

assert math.pi != mymath.pi

Note how definitions within a module can access other definitions within the module without having to qualify the name:

```python
def area(r):
    return pi * r * r
```

If you want to, you can also ask for names from a module to be imported so that you don't need to qualify them:

In [6]:
from math import pi

pi

3.141592653589793

Note that the `area` function will still need to be invoked as `mymath.area`.

### Reloading a module

Especially when working with notebooks you might change the code of a module and then would like to bring in the new code. In those cases, retyping `import mymath` won't work because the module is already loaded.

Instead, to load a fresh version of the module you can restart the session/kernel, or you can use the `reload` function from the `importlib` module which provides an interface to the mechanisms behind importing modules.

In [7]:
import mymath, importlib
importlib.reload(mymath)

<module 'mymath' from '/home/ubuntu/Development/git-repos/side_projects/python-workbench/part_1-python-fundamentals/03_basics-deep-dive/07_modules-and-scoping-rules/mymath.py'>

As a summary:
+ A module is a file defining one or more Python objects.
+ If the name of the module file is `modulename.py`, the Python module's name will be `modulename`.
+ You can bring a module named `modulename` into use with `import modulename` statement. When doing so, objects defined in the module will be accesible using `modulename.objectname`.
+ Specific names from a module can be brought directly into the program using `from modulename import objectname`. When doing so, `objectname` will be accessible without your needing to prepend it with `modulename`. This is recommended for bringing names that are often used.

## The `import` statement

There are three variants of the `import` statement:

+ `import modulename`
+ `from modulename import name1, name2, name3, ...`
+ `from modulename import *`

The last form brings into use all the exported names in `modulename`, that is, those that don't begin with an underscore `_`.
Also, if a list of names called `__all__` exists in the module or the package's `__init__.py`, those will be the names that are imported.

That last form might do not avert name clashing if two modules define the first name. In practice, you should use either of the two first forms of the `import` statement.

## Grokking the module search path

The variable `path` from the `sys` module tells you where exactly Python will look for modules:

In [8]:
import sys

sys.path

['/home/ubuntu/.pyenv/versions/3.12.5/lib/python312.zip',
 '/home/ubuntu/.pyenv/versions/3.12.5/lib/python3.12',
 '/home/ubuntu/.pyenv/versions/3.12.5/lib/python3.12/lib-dynload',
 '',
 '/home/ubuntu/Development/git-repos/side_projects/python-workbench/part_1-python-fundamentals/03_basics-deep-dive/07_modules-and-scoping-rules/.venv/lib/python3.12/site-packages']

The `sys.path` is a list of directories that Python will search in order when attempting to execute an `import` statement.

This variable is initialized from the value of the OS environment variable `PYTHONPATH`, if it exists, or from a default value that's dependent on your installation.

In addition, the `sys.path` variable has the directory containing the script inserted as its first element, which provides a convenient way of determining where the executing Python program is located. That is, when you do `python app.py`, `sys.path[0]` will tell you where `app.py` is located.

### Where to place your own modules

To ensure that your programs can use the modules you coded you need to:
+ Place your modules in one of the directories that Python normally searches for modules (not recommended, as this is intended for site-specific modules, that is, modules specific to your machine).
+ Place all the modules use by a Python program in the same directory as the program (good option for modules that are associated with a particular program).
+ Create a directory or directories to hold your modules and modify the `sys.path` variable so that it includes this new directory (good option for reusable modules).

For the third option, you'll have to set the `PYTHONPATH` environment variable, or add the directory to the default search paths using a `.pth` file.

### Private names in modules

We already established that identifiers in the module beginning with `_` are not imported by `from modulename import *`.


That way, module writers can ensure that internal names are not imported.

Consider the following module:

```python
"""modtest: test module with private and public identifiers"""
def f(x):
    return x

def _g(x):
    return x

a = 4
_b = 2
```

Neither `_g` nor `_b` will be imported using `from modtest import *`:

In [11]:
from modtest import *

try:
    _g(3)
except Exception as e:
    print("Oops:", {e})

try:
    _b
except Exception as e:
    print("Oops:", {e})

Oops: {NameError("name '_g' is not defined")}
Oops: {NameError("name '_b' is not defined")}


This convention of leading underscores to indicate private names is used throughout Python, not just in modules.

## Scoping rules and namespaces

A namespace in Python is a mapping from identifiers to objects.

When Python finds a statement like:

```python
x = 1
```

Python will add `x` to the namespace (if not already there) and associates it with the value `1`.

Python manages three namespaces: **local**, **global**, and **built-in**.

When an identifier is to be evaluated, Python first looks in the **local** namespace trying to locate functions or variables matching the identifier. If not found there, it checks the **global** namespace then. If it still hasn't been found, Python searches the **built-in** namespace. If not found, a `NameError` exception occurs.

For code within a module, a command executed in an interactive session, or code within a file (outside any function), the global and local namespaces are the same.

But when a function call is made, a local namespace is created, and a binding is entered in it for each parameter of the call. When a new variable definition is found within the function, a new binding will be creted within the function.

The global namespace of a function is the global namespace of the containing block of the function. This block is independent of the dynamic context from which it's called.

The built-in namespace is that of the `__builtins__` module. This module contains all the built-in functions such as `len`, `min`, `max`, etc., as well other built-in objects and classes such as `NameError`.

Because of how Python searches through the namespace (first local, then global, and finally built-in) you can override items in the built-in module.

For example, if you inadvertently create a variable named `list` you will be overriding the `list` function and won't be available anymore.

In [1]:
my_tuple = (1, 2)
print(list(my_tuple)) # not overridden yet

try:
    list = [1, 2]
    print(list(my_tuple)) # list object is not callable
except Exception as e:
    print("Oops:", {e})

[1, 2]
Oops: {TypeError("'list' object is not callable")}


The local and global namespaces can be accessed through the `locals()` and `globals()`

In [2]:
locals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'my_tuple = (1, 2)\nprint(list(my_tuple)) # not overridden yet\n\ntry:\n    list = [1, 2]\n    print(list(my_tuple)) # list object is not callable\nexcept Exception as e:\n    print("Oops:", {e})',
  'locals()'],
 '_oh': {},
 '_dh': [PosixPath('/home/ubuntu/Development/git-repos/side_projects/python-workbench/part_1-python-fundamentals/03_basics-deep-dive/07_modules-and-scoping-rules')],
 'In': ['',
  'my_tuple = (1, 2)\nprint(list(my_tuple)) # not overridden yet\n\ntry:\n    list = [1, 2]\n    print(list(my_tuple)) # list object is not callable\nexcept Exception as e:\n    print("Oops:", {e})',
  'locals()'],
 'Out': {},
 'get_ipython': <bound method InteractiveShell.get_ipython of <ipykernel.zmqshell.ZMQIntera

In [3]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'my_tuple = (1, 2)\nprint(list(my_tuple)) # not overridden yet\n\ntry:\n    list = [1, 2]\n    print(list(my_tuple)) # list object is not callable\nexcept Exception as e:\n    print("Oops:", {e})',
  'locals()',
  'globals()'],
 '_oh': {2: {...}},
 '_dh': [PosixPath('/home/ubuntu/Development/git-repos/side_projects/python-workbench/part_1-python-fundamentals/03_basics-deep-dive/07_modules-and-scoping-rules')],
 'In': ['',
  'my_tuple = (1, 2)\nprint(list(my_tuple)) # not overridden yet\n\ntry:\n    list = [1, 2]\n    print(list(my_tuple)) # list object is not callable\nexcept Exception as e:\n    print("Oops:", {e})',
  'locals()',
  'globals()'],
 'Out': {2: {...}},
 'get_ipython': <bound method InteractiveShel

See how the local and global namespaces for this notebook are the same.

If you continue creating variable and importing modules you'll see how different bindings are added to the namespaces:

In [4]:
z = 2
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'my_tuple = (1, 2)\nprint(list(my_tuple)) # not overridden yet\n\ntry:\n    list = [1, 2]\n    print(list(my_tuple)) # list object is not callable\nexcept Exception as e:\n    print("Oops:", {e})',
  'locals()',
  'globals()',
  'z = 2\nglobals()'],
 '_oh': {2: {...}, 3: {...}},
 '_dh': [PosixPath('/home/ubuntu/Development/git-repos/side_projects/python-workbench/part_1-python-fundamentals/03_basics-deep-dive/07_modules-and-scoping-rules')],
 'In': ['',
  'my_tuple = (1, 2)\nprint(list(my_tuple)) # not overridden yet\n\ntry:\n    list = [1, 2]\n    print(list(my_tuple)) # list object is not callable\nexcept Exception as e:\n    print("Oops:", {e})',
  'locals()',
  'globals()',
  'z = 2\nglobals()'],
 'Out': {2:

In [5]:
import math
from cmath import cos

globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'my_tuple = (1, 2)\nprint(list(my_tuple)) # not overridden yet\n\ntry:\n    list = [1, 2]\n    print(list(my_tuple)) # list object is not callable\nexcept Exception as e:\n    print("Oops:", {e})',
  'locals()',
  'globals()',
  'z = 2\nglobals()',
  'import math\nfrom cmath import cos\n\nglobals()'],
 '_oh': {2: {...}, 3: {...}, 4: {...}},
 '_dh': [PosixPath('/home/ubuntu/Development/git-repos/side_projects/python-workbench/part_1-python-fundamentals/03_basics-deep-dive/07_modules-and-scoping-rules')],
 'In': ['',
  'my_tuple = (1, 2)\nprint(list(my_tuple)) # not overridden yet\n\ntry:\n    list = [1, 2]\n    print(list(my_tuple)) # list object is not callable\nexcept Exception as e:\n    print("Oops:", {e})',


You can use the `del` statement to remove entries from the namespaces:

In [7]:
print(z)

del z

try:
    z
except Exception as e:
    print("Oops:", {e})

2
Oops: {NameError("name 'z' is not defined")}


This technique can be useful specially in the interactive mode to unload a previously loaded module or remove a previously made definition.

Let's try to inspect the local namespace when you define a function:

In [10]:
print(locals())

def f(x):
    print("On entry:", locals())
    y = x
    print("On exit", locals())

z = 2
f(z)

{'__name__': '__main__', '__doc__': 'Automatically created module for IPython interactive environment', '__package__': None, '__loader__': None, '__spec__': None, '__builtin__': <module 'builtins' (built-in)>, '__builtins__': <module 'builtins' (built-in)>, '_ih': ['', 'my_tuple = (1, 2)\nprint(list(my_tuple)) # not overridden yet\n\ntry:\n    list = [1, 2]\n    print(list(my_tuple)) # list object is not callable\nexcept Exception as e:\n    print("Oops:", {e})', 'locals()', 'globals()', 'z = 2\nglobals()', 'import math\nfrom cmath import cos\n\nglobals()', 'print(z)', 'print(z)\n\ndel z\n\ntry:\n    z\nexcept Exception as e:\n    print("Oops:", {e})', 'print(locals())\n\ndef f(x):\n    print("On entry:", locals)\n    y = x\n    print("On exit", locals)', 'print(locals())\n\ndef f(x):\n    print("On entry:", locals)\n    y = x\n    print("On exit", locals)\n\nz = 2\nf(z)', 'print(locals())\n\ndef f(x):\n    print("On entry:", locals())\n    y = x\n    print("On exit", locals())\n\nz = 

See how when you are within the function, the local namespace only reflects the variables that are accessible in the function.

To inspect the built-in namespace you need to use a different technique:

In [11]:
dir(__builtins__)

['ArithmeticError',
 'AssertionError',
 'AttributeError',
 'BaseException',
 'BaseExceptionGroup',
 'BlockingIOError',
 'BrokenPipeError',
 'BufferError',
 'BytesWarning',
 'ChildProcessError',
 'ConnectionAbortedError',
 'ConnectionError',
 'ConnectionRefusedError',
 'ConnectionResetError',
 'DeprecationWarning',
 'EOFError',
 'Ellipsis',
 'EncodingWarning',
 'EnvironmentError',
 'Exception',
 'ExceptionGroup',
 'False',
 'FileExistsError',
 'FileNotFoundError',
 'FloatingPointError',
 'FutureWarning',
 'GeneratorExit',
 'IOError',
 'ImportError',
 'ImportWarning',
 'IndentationError',
 'IndexError',
 'InterruptedError',
 'IsADirectoryError',
 'KeyError',
 'KeyboardInterrupt',
 'LookupError',
 'MemoryError',
 'ModuleNotFoundError',
 'NameError',
 'None',
 'NotADirectoryError',
 'NotImplemented',
 'NotImplementedError',
 'OSError',
 'OverflowError',
 'PendingDeprecationWarning',
 'PermissionError',
 'ProcessLookupError',
 'RecursionError',
 'ReferenceError',
 'ResourceWarning',
 'Runti

Given a module, `dir` returns the names defined in it.

If you call it without any argument, it will return a sorted list of identifiers in the local namespace:

In [12]:
x1 = 6
dir()

['In',
 'Out',
 '_',
 '_11',
 '_2',
 '_3',
 '_4',
 '_5',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '__vsc_ipynb_file__',
 '_dh',
 '_i',
 '_i1',
 '_i10',
 '_i11',
 '_i12',
 '_i2',
 '_i3',
 '_i4',
 '_i5',
 '_i6',
 '_i7',
 '_i8',
 '_i9',
 '_ih',
 '_ii',
 '_iii',
 '_oh',
 'cos',
 'exit',
 'f',
 'get_ipython',
 'list',
 'math',
 'my_tuple',
 'open',
 'quit',
 'x1',
 'z']